In [1]:
import json
import numpy as np
import pandas as pd
import multiprocessing

from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.core.problem import ElementwiseProblem, StarmapParallelization
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.pntx import SinglePointCrossover

In [2]:
# Read CSVs
df_classrooms = pd.read_csv("./datasets/clean/CaracterizaçãoDasSalasSmall.csv", sep=";", header=0)
df_classes = pd.read_csv("./datasets/clean/HorarioDeExemploSmall.csv", sep=";", header=0)

def get_characteristics(row):
  cols_with_x = [col for col in row.index if row[col] == "X"]
  return ", ".join(cols_with_x)

# Prepare df_classrooms
df_classrooms["caracteristicas"] = df_classrooms.apply(get_characteristics, axis=1)
df_classrooms = df_classrooms[["Nome sala", "Capacidade Normal", "caracteristicas"]]
df_classrooms = df_classrooms.rename(columns={"Nome sala":"Sala da aula", "Capacidade Normal":"Lotação", "caracteristicas":"Características reais da sala"})
print("df_classrooms len: ", len(df_classrooms))

# Prepare df_classes
df_classes["Sala da aula"] = ""
df_classes["Lotação"] = ""
df_classes["Características reais da sala"] = ""
print("df_classes len: ", len(df_classes))

df_classrooms len:  85
df_classes len:  5746


In [3]:
class ISCTE_CLASSES(ElementwiseProblem):
  def __init__(self, df_classes, df_classrooms, **kwargs):
    self.df_classes = df_classes
    self.df_classrooms = df_classrooms

    n_var = len(df_classes)

    xl = np.zeros(n_var)
    xu = np.array([len(df_classrooms) - 1] * len(df_classes))

    super().__init__(n_var=n_var, n_obj=3, xl=xl, xu=xu, **kwargs)


  def _evaluate(self, x, out, *args, **kwargs):
    df_schedule = self.arr_to_df(x)
    schedule_eval = self.df_evaluate(df_schedule)

    out["F"] = np.array(schedule_eval)


  def arr_to_df(self, arr_schedule):
    df_schedule = self.df_classes.copy()

    for i, row in df_schedule.iterrows():
      if arr_schedule[i] >= 97:
        print(arr_schedule)

      classroom = self.df_classrooms.iloc[int(arr_schedule[i])]

      df_schedule.at[i, "Sala da aula"] = classroom.iloc[0]
      df_schedule.at[i, "Lotação"] = classroom.iloc[1]
      df_schedule.at[i, "Características reais da sala"] = classroom.iloc[2]

    df_schedule["Datetime Início"] = pd.to_datetime(df_schedule['Dia'] + ' ' + df_schedule['Início'], format='%d/%m/%Y %H:%M:%S')
    df_schedule["Datetime Fim"] = pd.to_datetime(df_schedule['Dia'] + ' ' + df_schedule['Fim'], format='%d/%m/%Y %H:%M:%S')
    df_schedule["Lotação - Inscritos no turno"] = df_schedule.apply(lambda x: int(x["Lotação"]) - int(x["Inscritos no turno"]), axis=1)
    # df_schedule["Lotação 20% - Inscritos no turno"] = df_schedule.apply(lambda x: int(int(x["Lotação"]) * 1.2) - int(x["Inscritos no turno"]), axis=1)
    
    return df_schedule


  def check_overlap(self, start, end, df):
    for _, row in df.iterrows():
      tmp_start = row["Datetime Início"]
      tmp_end = row["Datetime Fim"]

      if tmp_start <= start < tmp_end or tmp_start < end <= tmp_end:
        return True
      
    return False
  
  # def check_overlap_2(self, start, end, df):
  #   overlaps = ((df['Datetime Início'] <= start) & (start < df['Datetime Fim'])) | ((df['Datetime Início'] < end) & (end <= df['Datetime Fim']))

  #   val_count = overlaps.value_counts()

  #   return True if val_count.get(True, 0) > 0 else False


  def df_evaluate(self, df):
    sobreposicao = 0
    # sobreposicao2 = 0
    superlotacao = (df["Lotação - Inscritos no turno"] < 0).sum()
    # superlotacao20 = (df["Lotação 20% - Inscritos no turno"] < 0).sum()
    caracteristicas = df.apply(lambda row: row["Características da sala pedida para a aula"] not in row["Características reais da sala"], axis=1).sum()

    for i, row in df.iterrows():
      tmp_df = df.loc[i + 1:].copy()
      tmp_df = tmp_df[(tmp_df['Sala da aula'] == row['Sala da aula']) & (tmp_df['Dia'] == row['Dia'])].copy()

      if tmp_df.shape[0] > 0:
        # if self.check_overlap_2(row["Datetime Início"], row["Datetime Fim"], tmp_df):
          # sobreposicao2 += 1
        if self.check_overlap(row["Datetime Início"], row["Datetime Fim"], tmp_df):
          sobreposicao += 1

    return [sobreposicao, superlotacao, caracteristicas]

In [4]:
# the number of process to be used
n_process = 6

# initialize the pool
pool = multiprocessing.Pool(n_process)

runner = StarmapParallelization(pool.starmap)

iscte_problem = ISCTE_CLASSES(df_classes, df_classrooms, elementwise_runner=runner)

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=13) # 13

nsga3 = NSGA3(
  pop_size=105, # 105
  ref_dirs=ref_dirs,
  sampling=IntegerRandomSampling(),
  crossover=SinglePointCrossover(prob=1),
  mutation=PolynomialMutation(prob=1, eta=10, repair=RoundingRepair()),
  eliminate_duplicates=True
)

iscte_res = minimize(
  iscte_problem,
  nsga3,
  seed=7,
  save_history=True,
  verbose=True,
  termination=('n_eval', 10500) # 315
)

print('Threads:', iscte_res.exec_time)

X = iscte_res.opt.get("X")
F = iscte_res.opt.get("F")

# print(X)
print(F)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      105 |      9 |             - |             -
     2 |      210 |     11 |  0.0112359551 |         ideal
     3 |      315 |     14 |  0.1111111111 |         ideal
     4 |      420 |     15 |  0.0909090909 |         ideal
     5 |      525 |     11 |  0.2031250000 |         ideal
     6 |      630 |     10 |  0.3140495868 |         ideal
     7 |      735 |     15 |  0.1509433962 |         ideal
     8 |      840 |     15 |  0.1093750000 |         ideal
     9 |      945 |     19 |  0.0714285714 |         ideal
    10 |     1050 |     19 |  0.0930232558 |         nadir
    11 |     1155 |     18 |  0.3437500000 |         nadir
    12 |     1260 |     18 |  0.0465116279 |         ideal
    13 |     1365 |     17 |  0.0833333333 |         ideal
    14 |     1470 |     24 |  0.1223021583 |         ideal
    15 |     1575 |     26 |  0.1250000000 |         ideal
    16 |     1680 |     28 |  0.0760869565 |         nad

In [5]:
pool.close()

In [6]:
print(iscte_res.history[0].opt.get("F"))

[[ 481. 1273. 2445.]
 [ 474. 1303. 2434.]
 [ 496. 1240. 2483.]
 [ 460. 1326. 2503.]
 [ 476. 1283. 2397.]
 [ 462. 1325. 2518.]
 [ 481. 1263. 2462.]
 [ 499. 1251. 2423.]
 [ 463. 1295. 2473.]]


In [7]:
for i in [0, 9, 24, 49, 74, 99]:
  score_json = json.dumps(iscte_res.history[i].opt.get("F").tolist(), indent=4)
  schedule_json = json.dumps(iscte_res.history[i].opt.get("X").tolist(), indent=4)

  with open(f"./results/final/NSGA_III/score_gen_{i+1}.json", "w") as outfile:
    outfile.write(score_json)
    
  with open(f"./results/final/NSGA_III/schedule_gen_{i+1}.json", "w") as outfile:
    outfile.write(schedule_json)

In [8]:
# result_dict = {"Score": F.tolist(), "Schedule": X.tolist()}

# results_json = json.dumps(result_dict, indent=4)

# with open("./results/NSGA_III_result_2_1.json", "w") as outfile:
#   outfile.write(results_json)